# Data Preparation
In this notebook, I will create the training data file to train a model to do one-step forecasting.  The idea is to create a file that is a superset of features.  This would allow choosing different subsets of features to used for training.

We start with the provided daily sales count file.

In [27]:
import numpy as np
import pandas as pd

transactions = pd.read_csv('data/provided/sales_train_v2.csv')
transactions.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


The daily sales counts need to be aggregated into monthly counts because we will be predicting monthly sales counts.

In [28]:
# aggregate records by date, shop and item
monthly_totals = transactions.groupby(['date_block_num', 'shop_id', 'item_id'],as_index=False)[['item_cnt_day']].sum()
monthly_totals.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


Each record is a sales count for a particular month (let's call it the base month).  For each record, let's append the sales counts for the months prior to the base month.  To do so, we need shift the copy the monthly totals, shift the date_block_num values for the copied totals accordingly and then merge the two totals.

In [29]:
# this is a helper function to append the monthly sales count from a prior month

# return a copy of the monthly totals with the date_block_num shifted by an index
def get_shifted_monthly_totals(df, month_idx):
    totals = df.copy()
    totals['date_block_num'] = totals['date_block_num'] - month_idx
    return totals

df = get_shifted_monthly_totals(monthly_totals, -1)
df.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,1,0,32,6.0
1,1,0,33,3.0
2,1,0,35,1.0
3,1,0,43,1.0
4,1,0,51,2.0


In [30]:
#rename the item_cnt_day field to t (base month) for better clarity
training_data = monthly_totals.copy()
training_data.rename(index=str, columns={"item_cnt_day":"t"}, inplace=True)
training_data.head()

,date_block_num,shop_id,item_id,t
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


We will start with adding the sales count for the prior month and then append the sales count from two months ago. 

In [31]:
# add previous month's sales count to training data for each record
prior_totals = get_shifted_monthly_totals(monthly_totals, -1)
training_data = training_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'item_id'], how='left')
training_data.rename(index=str, columns={"item_cnt_day":"t-1"}, inplace=True)
training_data.tail()

,date_block_num,shop_id,item_id,t,t-1
1609119,33,59,22087,6.0,3.0
1609120,33,59,22088,2.0,1.0
1609121,33,59,22091,1.0,3.0
1609122,33,59,22100,1.0,1.0
1609123,33,59,22102,1.0,NaN


In [32]:
# add sales count from 2 months ago to training data for each record
prior_totals = get_shifted_monthly_totals(monthly_totals, -2)
training_data = training_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'item_id'], how='left')
training_data.rename(index=str, columns={"item_cnt_day":"t-2"}, inplace=True)
training_data.tail()

,date_block_num,shop_id,item_id,t,t-1,t-2
1609119,33,59,22087,6.0,3.0,2.0
1609120,33,59,22088,2.0,1.0,7.0
1609121,33,59,22091,1.0,3.0,NaN
1609122,33,59,22100,1.0,1.0,NaN
1609123,33,59,22102,1.0,NaN,2.0


Now, we will append the sales count for 5 months ago and 11 months ago.  We are attempting to predict the sales count for the month after the base month (or t+1).  Therefore, the sales counts for 5 months and 11 months back of the base month are the sales counts for 6 months and 12 months before the target month.

In [33]:
# add sales count from 5 months ago to training data for each record
prior_totals = get_shifted_monthly_totals(monthly_totals, -5)
training_data = training_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'item_id'], how='left')
training_data.rename(index=str, columns={"item_cnt_day":"t-5"}, inplace=True)
training_data.tail()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5
1609119,33,59,22087,6.0,3.0,2.0,2.0
1609120,33,59,22088,2.0,1.0,7.0,3.0
1609121,33,59,22091,1.0,3.0,NaN,NaN
1609122,33,59,22100,1.0,1.0,NaN,NaN
1609123,33,59,22102,1.0,NaN,2.0,NaN


In [34]:
# add sales count from 11 months ago to training data for each record
prior_totals = get_shifted_monthly_totals(monthly_totals, -11)
training_data = training_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'item_id'], how='left')
training_data.rename(index=str, columns={"item_cnt_day":"t-11"}, inplace=True)
training_data.tail()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11
1609119,33,59,22087,6.0,3.0,2.0,2.0,6.0
1609120,33,59,22088,2.0,1.0,7.0,3.0,15.0
1609121,33,59,22091,1.0,3.0,NaN,NaN,1.0
1609122,33,59,22100,1.0,1.0,NaN,NaN,NaN
1609123,33,59,22102,1.0,NaN,2.0,NaN,NaN


It's time to append the target variable for each record.

In [35]:
# add sales count for following month to training data for each record
next_totals = get_shifted_monthly_totals(monthly_totals, 1)
training_data = training_data.merge(next_totals, on=['date_block_num', 'shop_id', 'item_id'], how='left')
training_data.rename(index=str, columns={"item_cnt_day":"t+1"}, inplace=True)
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1
0,0,0,32,6.0,NaN,NaN,NaN,NaN,10.0
1,0,0,33,3.0,NaN,NaN,NaN,NaN,3.0
2,0,0,35,1.0,NaN,NaN,NaN,NaN,14.0
3,0,0,43,1.0,NaN,NaN,NaN,NaN,NaN
4,0,0,51,2.0,NaN,NaN,NaN,NaN,3.0


In [36]:
# save file in case we need to want to start with the data up to this point
print(training_data.shape)
training_data.to_csv('data/train_item_prior_y.csv', index=False)

(1609124, 9)


Now, we need to remove records for the year 2013 because their t-12 value is invalid.

In [37]:
# only records from the year 2014 onwards can be used for training
training_data = training_data.loc[(training_data['date_block_num'] >= 12)]
print(training_data.shape)

# replace NaN with 0.0
training_data.fillna(0.0, inplace=True)
training_data.head()

(921400, 9)


,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1
687724,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0
687725,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0
687726,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0
687727,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0
687728,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0


At this point, we have 8 features (and the target variable) and over 900000 records in the training file.

### Categories
Let's add features related to the item category.  We will build a separate file with just category features and then merge it with the exsiting training file.  We start by adding the category id for each respective item for each record in the monthly sales counts for items.

In [38]:
# mix in item_category_id from provided reference file
items = pd.read_csv('data/provided/items.csv')
monthly_categ_totals = monthly_totals.merge(items, on=['item_id'], how='left').drop('item_name', axis=1)
monthly_categ_totals.head()

,date_block_num,shop_id,item_id,item_cnt_day,item_category_id
0,0,0,32,6.0,40
1,0,0,33,3.0,37
2,0,0,35,1.0,40
3,0,0,43,1.0,40
4,0,0,51,2.0,57


We don't have any other provided information for the categories.  So let's add sales counts of prior months for each respective shop and category pairing.  Hopefully, this is information that can help a model determine the 'type' of category the item belongs to. 

In [39]:
# aggregate records based on date, shop, category
monthly_categ_totals = monthly_categ_totals.groupby(['date_block_num', 'shop_id', 'item_category_id'],as_index=False)[['item_cnt_day']].sum()
monthly_categ_totals.rename(index=str, columns={"item_category_id":"categ_id"}, inplace=True)
monthly_categ_totals.head()

,date_block_num,shop_id,categ_id,item_cnt_day
0,0,0,2,53.0
1,0,0,3,28.0
2,0,0,4,16.0
3,0,0,5,28.0
4,0,0,6,65.0


In [40]:
# rename the count for the base month to categ_t for easier comparison
categ_data = monthly_categ_totals.copy()
categ_data.rename(index=str, columns={"item_cnt_day":"categ_t"}, inplace=True)
categ_data.head()

,date_block_num,shop_id,categ_id,categ_t
0,0,0,2,53.0
1,0,0,3,28.0
2,0,0,4,16.0
3,0,0,5,28.0
4,0,0,6,65.0


In [41]:
# add sales counts from previous month
prior_totals = get_shifted_monthly_totals(monthly_categ_totals, -1)
categ_data = categ_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'categ_id'], how='left')
categ_data.rename(index=str, columns={"item_cnt_day":"categ_t-1"}, inplace=True)

# add sales counts from 11 months ago
prior_totals = get_shifted_monthly_totals(monthly_categ_totals, -11)
categ_data = categ_data.merge(prior_totals, on=['date_block_num', 'shop_id', 'categ_id'], how='left')
categ_data.rename(index=str, columns={"item_cnt_day":"categ_t-11"}, inplace=True)

# remove records from the year 2013
categ_data = categ_data.loc[(categ_data['date_block_num'] >= 12)]

# replace invalid NaN values with 0.0
categ_data.fillna(0.0, inplace=True)
categ_data.head()

,date_block_num,shop_id,categ_id,categ_t,categ_t-1,categ_t-11
22290,12,2,2,23.0,35.0,21.0
22291,12,2,3,15.0,50.0,6.0
22292,12,2,6,18.0,24.0,8.0
22293,12,2,11,5.0,17.0,2.0
22294,12,2,15,2.0,5.0,2.0


Now, let's merge the category features into the existing training file.

In [42]:
# Using reference file to add the item_category_id for each record in the training file
training_data = training_data.merge(items, on=['item_id'], how='left').drop('item_name', axis=1)
training_data.rename(index=str, columns={"item_category_id":"categ_id"}, inplace=True)
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id
0,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0,40
1,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0,37
2,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0,37
3,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0,73
4,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0,73


In [43]:
# merge the file with category features into the training file
training_data = training_data.merge(categ_data, on=['date_block_num', 'shop_id', 'categ_id'], how='left')
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id,categ_t,categ_t-1,categ_t-11
0,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0,40,76.0,93.0,40.0
1,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0,37,44.0,55.0,21.0
2,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0,37,44.0,55.0,21.0
3,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0,73,4.0,3.0,10.0
4,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0,73,4.0,3.0,10.0


At this point, the training file has 12 features, plus the target variable.

### Shops
We will go through the same process for shops that we went through with categories: build a file with features related to shops and then merge these features into the training file.

We start by aggregating the records of sales counts by shop and category that we derived in the last section so we have monthly sales counts by shop.

In [44]:
# aggregate records based on date, shop
monthly_shop_totals = monthly_categ_totals.groupby(['date_block_num', 'shop_id'],as_index=False)[['item_cnt_day']].sum()
monthly_shop_totals.head()

,date_block_num,shop_id,item_cnt_day
0,0,0,5578.0
1,0,1,2947.0
2,0,2,1146.0
3,0,3,767.0
4,0,4,2114.0


In [45]:
# once again, rename the count field for easier comparison
shop_data = monthly_shop_totals.copy()
shop_data.rename(index=str, columns={"item_cnt_day":"shop_t"}, inplace=True)
shop_data.head()

,date_block_num,shop_id,shop_t
0,0,0,5578.0
1,0,1,2947.0
2,0,2,1146.0
3,0,3,767.0
4,0,4,2114.0


In [46]:
# add sales counts from previous month
prior_totals = get_shifted_monthly_totals(monthly_shop_totals, -1)
shop_data = shop_data.merge(prior_totals, on=['date_block_num', 'shop_id'], how='left')
shop_data.rename(index=str, columns={"item_cnt_day":"shop_t-1"}, inplace=True)

# add sales counts from 11 months ago
prior_totals = get_shifted_monthly_totals(monthly_shop_totals, -11)
shop_data = shop_data.merge(prior_totals, on=['date_block_num', 'shop_id'], how='left')
shop_data.rename(index=str, columns={"item_cnt_day":"shop_t-11"}, inplace=True)

# remove records for the year 2013
shop_data = shop_data.loc[(shop_data['date_block_num'] >= 12)]

# replace invalid NaN values with 0.0
shop_data.fillna(0.0, inplace=True)
shop_data.head()

,date_block_num,shop_id,shop_t,shop_t-1,shop_t-11
547,12,2,890.0,1322.0,488.0
548,12,3,968.0,1134.0,798.0
549,12,4,1430.0,2248.0,2025.0
550,12,5,1639.0,2223.0,877.0
551,12,6,3024.0,5467.0,4007.0


Merge the file of shop features into the training file.

In [47]:
training_data = training_data.merge(shop_data, on=['date_block_num', 'shop_id'], how='left')
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id,categ_t,categ_t-1,categ_t-11,shop_t,shop_t-1,shop_t-11
0,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0,40,76.0,93.0,40.0,890.0,1322.0,488.0
1,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0
2,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0
3,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0
4,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0


There are now 15 features in the training file.

### Month and Year
We currently have a feature called date_block_num in the training file.  To help a model learn the cyclical sales patterns each year, we need to replace this feature with a year and a month that corresponds to it.

The month will be indexed from 0 so that January is 0, February is 1, and so on.

In [48]:
# helper function to extract the year from a date block number
def get_year_from_date_block_num(date_block_num):
    if date_block_num < 12:
        return 2013
    elif date_block_num < 24:
        return 2014
    else:
        return 2015

# helper function to extract the month from a date block number
def get_month_from_date_block_num(date_block_num):
    return int(date_block_num % 12)

print('date block: 11 =' , get_year_from_date_block_num(11), get_month_from_date_block_num(11))
print('date block: 33 =' , get_year_from_date_block_num(33), get_month_from_date_block_num(33))


date block: 11 = 2013 11
date block: 33 = 2015 9


In [49]:
def get_year_for_row(row):
    return get_year_from_date_block_num(row['date_block_num'])

# extract the year from the date block number and add it as a field to each record in the training file
training_data['year'] = training_data.apply(lambda row: get_year_for_row(row), axis=1)
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id,categ_t,categ_t-1,categ_t-11,shop_t,shop_t-1,shop_t-11,year
0,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0,40,76.0,93.0,40.0,890.0,1322.0,488.0,2014
1,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0,2014
2,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0,2014
3,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0,2014
4,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0,2014


In [50]:
def get_month_for_row(row):
    return get_month_from_date_block_num(row['date_block_num'])

# extract the month from the date block number and add it as a field to each record in the training file
training_data['month'] = training_data.apply(lambda row: get_month_for_row(row), axis=1)
training_data.head()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id,categ_t,categ_t-1,categ_t-11,shop_t,shop_t-1,shop_t-11,year,month
0,12,2,32,1.0,0.0,0.0,0.0,0.0,0.0,40,76.0,93.0,40.0,890.0,1322.0,488.0,2014,0
1,12,2,33,1.0,1.0,2.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0,2014,0
2,12,2,99,1.0,0.0,0.0,0.0,0.0,0.0,37,44.0,55.0,21.0,890.0,1322.0,488.0,2014,0
3,12,2,482,2.0,1.0,2.0,0.0,1.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0,2014,0
4,12,2,485,1.0,1.0,0.0,0.0,0.0,1.0,73,4.0,3.0,10.0,890.0,1322.0,488.0,2014,0


We will save the training file because it has records for date_block_num 33.  We need these records to merge into the pairings in the provided test file in order to generate the required predictions for the Kaggle competition.

In [51]:
training_data.to_csv('data/td_all.csv', index=False)
training_data.tail()

,date_block_num,shop_id,item_id,t,t-1,t-2,t-5,t-11,t+1,categ_id,categ_t,categ_t-1,categ_t-11,shop_t,shop_t-1,shop_t-11,year,month
921395,33,59,22087,6.0,3.0,2.0,2.0,6.0,0.0,83,9.0,7.0,25.0,790.0,914.0,1610.0,2015,9
921396,33,59,22088,2.0,1.0,7.0,3.0,15.0,0.0,83,9.0,7.0,25.0,790.0,914.0,1610.0,2015,9
921397,33,59,22091,1.0,3.0,0.0,0.0,1.0,0.0,83,9.0,7.0,25.0,790.0,914.0,1610.0,2015,9
921398,33,59,22100,1.0,1.0,0.0,0.0,0.0,0.0,42,3.0,5.0,0.0,790.0,914.0,1610.0,2015,9
921399,33,59,22102,1.0,0.0,2.0,0.0,0.0,0.0,42,3.0,5.0,0.0,790.0,914.0,1610.0,2015,9


Now, let's do some final cleanup on the training file before we save it.

In [52]:
# remove records with date_block_num 33 (because we don't have valid t+1 values for this date block)
training_data = training_data.loc[(training_data['date_block_num'] < 33)]

# drop the date_block_num field and order the features in the form of X, y
training_data = training_data[['year','month','shop_id','shop_t','shop_t-1','shop_t-11','categ_id','categ_t','categ_t-1','categ_t-11','item_id','t','t-1','t-2','t-5','t-11','t+1']]

print(training_data.shape)
training_data.to_csv('data/training_data.csv', index=False)
training_data.tail()

(889869, 17)


,year,month,shop_id,shop_t,shop_t-1,shop_t-11,categ_id,categ_t,categ_t-1,categ_t-11,item_id,t,t-1,t-2,t-5,t-11,t+1
889864,2015,8,59,914.0,1214.0,1211.0,83,7.0,10.0,16.0,22087,3.0,2.0,5.0,2.0,6.0,6.0
889865,2015,8,59,914.0,1214.0,1211.0,83,7.0,10.0,16.0,22088,1.0,7.0,7.0,3.0,7.0,2.0
889866,2015,8,59,914.0,1214.0,1211.0,83,7.0,10.0,16.0,22091,3.0,0.0,0.0,0.0,1.0,1.0
889867,2015,8,59,914.0,1214.0,1211.0,42,5.0,5.0,0.0,22100,1.0,0.0,0.0,0.0,0.0,1.0
889868,2015,8,59,914.0,1214.0,1211.0,38,13.0,13.0,13.0,22139,1.0,0.0,0.0,0.0,0.0,0.0


We're done.  We have a suitable training file with almost 900000 records, each with 16 features plus the target variable.